<a href="https://colab.research.google.com/github/Hyewon97/python_demo/blob/main/p302_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91_%EC%A0%95%EC%A0%81_%EB%8F%84%EC%8B%9C%EB%B3%84_%EA%B8%B0%EC%83%81%EC%B2%AD_%EB%82%A0%EC%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 도시별 기상청 날씨 Crowling

https://www.weather.go.kr/w/obs-climate/land/city-obs.do

구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/dataanalysis/data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/dataanalysis/data


In [ ]:
!pwd

/content/drive/MyDrive/dataanalysis/data


###1 import

In [ ]:
from bs4 import BeautifulSoup
import requests

### 2 작업결과를 저장할 리스트 준비

In [ ]:
result =[]

3 BeatifulSoup 객체 생성하여 파

In [ ]:
del result[:]

header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
url= "https://www.weather.go.kr/w/obs-climate/land/city-obs.do"

res = requests.get(url, headers = header)
soup = BeautifulSoup(res.text, 'html.parser')
print(type(soup))

#tbody_tag = soup.find('tbody', attrs={"aria-live":"polite"}) #attrs={"aria-live":"polite"}), 이런 속서은 가지고 오지 못한다.
tbody_tag = soup.find('tbody')
#print(tbody_tag)
tr_rs_tag = tbody_tag.find_all('tr')
# print(tr_rs_tag)
# print(type(tr_rs_tag))
print(tr_rs_tag[0])

for tr_tag in tr_rs_tag :
  td_rs_tag = tr_tag.find_all('td')
  city = td_rs_tag[0].string          # 지역
  cur_temp = td_rs_tag[5].string      # 현재기온
  sen_temp = td_rs_tag[7].string      # 체감온도
  result.append([city, cur_temp, sen_temp])

#result

<class 'bs4.BeautifulSoup'>
<tr>
<td><a href="city-obs.do?tm=2023.2.15.13:00&amp;type=t99&amp;mode=0&amp;reg=100&amp;auto_man=m&amp;stn=105">강릉</a></td>
<td> </td>
<td>2.7</td>
<td> </td>
<td> </td>
<td>3.1</td>
<td>0.4</td>
<td>0.7</td>
<td>13.9</td>
<td>15.6</td>
<td>83</td>
<td>북북동</td>
<td><script>writeWindSpeed('2.4', false, '', '', 1);</script></td>
<td>1032.4</td>
</tr>


4.도시별, 현재기온, 체감온도 크롤링 결과

In [ ]:
for element in result:
  print(element)
  break

['강릉', '2.7', '0.8']


5. 저장할 날짜 지

In [ ]:
import datetime

to_now=datetime.datetime.now()
to_now = to_now.strftime('%Y-%m-%d %H:%M:%s')

6. 크롤링 결과를 csv 파일로 저장

In [ ]:
import csv

# fields names
fields =['City', 'CurTemp','SenTemp']

with open('./%s_weather.csv' % to_now, 'w',newline='',encoding='cp949') as f:
  write = csv.writer(f)
  write.writerow(fields)
  write.writerows(result)

7 weather.csv 파일 읽어오기

In [ ]:
with open('./%s_weather.csv' % to_now,'r', encoding='cp949') as f:
    reader = csv.reader(f)
    for row in reader:
      for cell in row:
        print(cell, end=',')
      print(end= '\n')
      break;
    # td_list = element.split(",")
    # print(td_list[0], td_list[1], td_list[2])

City,CurTemp,SenTemp,


### 8 weather.py 파일 생성

In [ ]:
%%writefile ./weatehr_collector.py

from bs4 import BeautifulSoup
import requests
import datetime
import csv

result =[]

def weatehr_process() :
  header={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}
  url= "https://www.weather.go.kr/w/obs-climate/land/city-obs.do"

  res = requests.get(url, headers = header)
  soup = BeautifulSoup(res.text, 'html.parser')
  print(type(soup))

  tbody_tag = soup.find('tbody')
  tr_rs_tag = tbody_tag.find_all('tr')
  print(tr_rs_tag[0])

  for tr_tag in tr_rs_tag :
    td_rs_tag = tr_tag.find_all('td')
    city = td_rs_tag[0].string          # 지역
    cur_temp = td_rs_tag[5].string      # 현재기온
    sen_temp = td_rs_tag[7].string      # 체감온도
    result.append([city, cur_temp, sen_temp])


def weatehr_sav(filename):
  fields =['City', 'CurTemp','SenTemp']

  with open('./%s-weather.csv' % filename, 'w',newline='',encoding='cp949') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerow(result)


def weather_load(filename):
  with open('./%s-weather.csv' % filename, 'r',encoding='cp949') as f:
      reader = csv.reader(f)
      for row in reader:
        for cell in row:
          print(cell, end=',')
        print(end='\n')

def main():
  weather_process()

  to_now=datetime.datetime.now()
  to_now = to_now.strftime('%Y-%m-%d %H:%M:%s')

  weatehr_save(to_now)
  weatehr_load(to_now)


if__name__=='__main__' :
  main()

Overwriting ./weatehr_collector.py
